In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense

# Rede Neural - Autoencoder
## Classificação

In [5]:
# Importando bade de dados
(previsores_treinamento, classe_treinamento), (previsores_teste, classe_teste) = mnist.load_data()

In [6]:
# Convertendo e normalizando dados
previsores_treinamento = previsores_treinamento.astype('float32') / 255
previsores_teste = previsores_teste.astype('float32') / 255

In [7]:
# Criando classe dummy
classe_dummy_treinamento = np_utils.to_categorical(classe_treinamento)
classe_dummy_teste = np_utils.to_categorical(classe_teste)

In [8]:
# Realisando redimencinamento 
previsores_treinamento = previsores_treinamento.reshape((len(previsores_treinamento), np.prod(previsores_treinamento.shape[1:])))
previsores_teste = previsores_teste.reshape((len(previsores_teste), np.prod(previsores_teste.shape[1:])))

In [9]:
# 784 - 32 - 784
# Criando estrutura da rede neural
autoencoder = Sequential()
autoencoder.add(Dense(units = 32, activation = 'relu', input_dim = 784))
autoencoder.add(Dense(units = 784, activation = 'sigmoid'))
# Compilando rede neural
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Treinamento e teste do modelo
autoencoder.fit(previsores_treinamento, previsores_treinamento, epochs = 50, batch_size = 256, validation_data = (previsores_teste, previsores_teste))

Epoch 1/50
235/235 [==============================] - 3s 11ms/step - loss: 0.3885 - accuracy: 0.0085 - val_loss: 0.1882 - val_accuracy: 0.0108
Epoch 2/50
235/235 [==============================] - 2s 9ms/step - loss: 0.1783 - accuracy: 0.0121 - val_loss: 0.1528 - val_accuracy: 0.0102
Epoch 3/50
235/235 [==============================] - 2s 9ms/step - loss: 0.1485 - accuracy: 0.0101 - val_loss: 0.1334 - val_accuracy: 0.0098
Epoch 4/50
235/235 [==============================] - 2s 8ms/step - loss: 0.1316 - accuracy: 0.0097 - val_loss: 0.1211 - val_accuracy: 0.0096
Epoch 5/50
235/235 [==============================] - 2s 9ms/step - loss: 0.1201 - accuracy: 0.0106 - val_loss: 0.1124 - val_accuracy: 0.0101
Epoch 6/50
235/235 [==============================] - 2s 9ms/step - loss: 0.1123 - accuracy: 0.0100 - val_loss: 0.1063 - val_accuracy: 0.0111
Epoch 7/50
235/235 [==============================] - 2s 9ms/step - loss: 0.1065 - accuracy: 0.0114 - val_loss: 0.1021 - val_accuracy: 0.0101
Epoch

In [10]:
# Encoder baseado no modelo autoencoder
dimensao_original = Input(shape=(784,))
camada_encoder = autoencoder.layers[0]
encoder = Model(dimensao_original, camada_encoder(dimensao_original))

In [11]:
# Testando enconder e decoder
previsores_treinamento_codificados = encoder.predict(previsores_treinamento)
previsores_teste_codificados = encoder.predict(previsores_teste)

In [12]:
# Testando sem redução de dimensionalidade
# Criando estrutura da rede neural
c1 = Sequential()
c1.add(Dense(units = 397, activation = 'relu', input_dim = 784))
c1.add(Dense(units = 397, activation = 'relu'))
c1.add(Dense(units = 10, activation = 'softmax'))
# Compilando rede neural
c1.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# Treinamento e teste do modelo
c1.fit(previsores_treinamento, classe_dummy_treinamento, batch_size = 256, epochs = 100, validation_data=(previsores_teste, classe_dummy_teste))

Epoch 1/100
235/235 [==============================] - 4s 14ms/step - loss: 0.5197 - accuracy: 0.8537 - val_loss: 0.1341 - val_accuracy: 0.9594
Epoch 2/100
235/235 [==============================] - 3s 14ms/step - loss: 0.1054 - accuracy: 0.9693 - val_loss: 0.0954 - val_accuracy: 0.9704
Epoch 3/100
235/235 [==============================] - 3s 12ms/step - loss: 0.0642 - accuracy: 0.9798 - val_loss: 0.0728 - val_accuracy: 0.9771
Epoch 4/100
235/235 [==============================] - 3s 12ms/step - loss: 0.0390 - accuracy: 0.9884 - val_loss: 0.0730 - val_accuracy: 0.9781
Epoch 5/100
235/235 [==============================] - 3s 15ms/step - loss: 0.0272 - accuracy: 0.9922 - val_loss: 0.0719 - val_accuracy: 0.9777
Epoch 6/100
235/235 [==============================] - 3s 14ms/step - loss: 0.0234 - accuracy: 0.9935 - val_loss: 0.0690 - val_accuracy: 0.9807
Epoch 7/100
235/235 [==============================] - 3s 15ms/step - loss: 0.0147 - accuracy: 0.9955 - val_loss: 0.0743 - val_accuracy:

In [13]:
# Testando com redução de dimensionalidade
# Criando estrutura da rede neural
c2 = Sequential()
c2.add(Dense(units = 21, activation = 'relu', input_dim = 32))
c2.add(Dense(units = 21, activation = 'relu'))
c2.add(Dense(units = 10, activation = 'softmax'))
# Compilando rede neural
c2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# Treinamento e teste do modelo
c2.fit(previsores_treinamento_codificados, classe_dummy_treinamento, batch_size = 256, epochs = 100, validation_data=(previsores_teste_codificados, classe_dummy_teste))

Epoch 1/100
235/235 [==============================] - 1s 3ms/step - loss: 2.0235 - accuracy: 0.3366 - val_loss: 0.8110 - val_accuracy: 0.7526
Epoch 2/100
235/235 [==============================] - 1s 2ms/step - loss: 0.7000 - accuracy: 0.7887 - val_loss: 0.5080 - val_accuracy: 0.8462
Epoch 3/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4905 - accuracy: 0.8536 - val_loss: 0.4262 - val_accuracy: 0.8699
Epoch 4/100
235/235 [==============================] - 1s 2ms/step - loss: 0.4174 - accuracy: 0.8752 - val_loss: 0.3765 - val_accuracy: 0.8868
Epoch 5/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3698 - accuracy: 0.8893 - val_loss: 0.3401 - val_accuracy: 0.8959
Epoch 6/100
235/235 [==============================] - 1s 2ms/step - loss: 0.3294 - accuracy: 0.9016 - val_loss: 0.3151 - val_accuracy: 0.9018
Epoch 7/100
235/235 [==============================] - 0s 2ms/step - loss: 0.3159 - accuracy: 0.9045 - val_loss: 0.3020 - val_accuracy: 0.9067